In [1]:
import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np
import pandas as pd

In [5]:
# 读取纽约市2017年的Land Cover数据集
land_cover_path = "E:\\Master Works\\Urban marchine learning\\Lab\\Land cover_2017_NYC.tif"
land_cover_data = rasterio.open(land_cover_path)

# 读取曼哈顿岛上不同社区区域的geojson文件
geojson_path = "E:\\Master Works\\Urban marchine learning\\Lab\\0.geojson"
communities = gpd.read_file(geojson_path)

# 检查CRS是否相同，如果不同则重新投影
if communities.crs != land_cover_data.crs:
    communities = communities.to_crs(land_cover_data.crs)

In [ ]:
# Land cover值与类型映射
value_mapping = {
    1: "Tree Canopy",
    2: "Grass/Shrubs",
    3: "Bare Soil",
    4: "Water",
    5: "Buildings",
    6: "Roads",
    7: "Other Impervious",
    8: "Railroads",
}

results = []

# 获取像素大小（平方米）
pixel_size = land_cover_data.transform[0] * land_cover_data.transform[4]


# 遍历每个社区
total_communities = len(communities)
for index, community in communities.iterrows():
    try:
        # 裁剪Land Cover数据集
        masked, transform = rasterio.mask.mask(land_cover_data, [community.geometry], crop=True)
        masked = masked[0]

        # 统计land cover类型
        unique, counts = np.unique(masked, return_counts=True)
        total_pixels = masked.size

        # 计算面积和百分比
        areas = dict(zip(unique, counts * pixel_size))
        percentages = dict(zip(unique, counts / total_pixels * 100))

        # 将像素值转换为land cover类型
        areas = {value_mapping.get(k, k): v for k, v in areas.items() if k != 0}
        percentages = {value_mapping.get(k, k): v for k, v in percentages.items() if k != 0}

        # 合并面积和百分比字典
        result = {f"{k} Area": v for k, v in areas.items()}
        result.update({f"{k} Percentage": v for k, v in percentages.items()})
        result["ElectDist"] = community["ElectDist"]

        results.append(result)

        # 显示进度百分比
        progress = (index + 1) / total_communities * 100
        print(f"Progress: {progress:.2f}%")
    except ValueError as e:
        print(f"Error for community '{community['ElectDist']}': {e}")


In [ ]:
# 输出结果到CSV文件
df = pd.DataFrame(results)
column_order = ["ElectDist"] + [f"{value_mapping[val]} {metric}" for val in sorted(value_mapping.keys()) for metric in ["Area", "Percentage"]]
df = df[column_order]
df.to_csv("land_cover_areas_percentages.csv", index=False)